#### Задача 1 (2 балла). 

Попробуем себя в решении задачи определения темы текста. Будем считать, что два текста похожи по теме, если у них больше общих слов (только не предлогов с союзами), чем у других текстов. У нашей программы для определения темы будет несколько готовых текстов (достаточно больших!) с уже известной темой в базе: выберите тексты (и темы) самостоятельно, 5-6 будет достаточно. 

Что должна делать программа? При запуске вы ей сообщаете название нового файла с текстом, который нужно классифицировать, она его открывает, обрабатывает и сравнивает с текстами в своей базе. С которым из текстов оказалось больше всего общих слов, того и тема! Очевидно, вам понадобится какие-то слова из текстов отбрасывать (подумайте, каким образом это сделать - здесь на самом деле несколько вариантов концепций), а еще лемматизировать или хотя бы использовать стемминг. 

Когда будете сдавать это задание, пожалуйста, пришлите и файлы с текстами. И имейте в виду, если тексты будут вставлены прямо в код и слишком короткие, я задачу засчитаю только вполовину. 

Для красоты можно 1) написать все в классах (вообще-то даже **нужно**) 2) использовать argparse (тогда код в .py)

In [1]:
import glob
import os
from textsim import TextSimilarity

Я также использую тексты таких тем, на которых программа не обучалась.

In [2]:
text_similarity = TextSimilarity()

path = os.path.join("test_texts", "*")

for file in glob.glob(path):
    file_name = os.path.basename(file)
    with open(file) as file:
        print(f"Анализация {file_name}")
        print(f"{text_similarity.get_theme(file.read())}\n")


Анализация радиоэлектроника.txt
Вероятная тема текста: базы данных.txt

Анализация архитектура.txt
Вероятная тема текста: экономика.txt

Анализация документальная литература.txt
Вероятная тема текста: научная фантастика.txt

Анализация экономика.txt
Вероятная тема текста: экономика.txt

Анализация базы данных.txt
Вероятная тема текста: базы данных.txt

Анализация домашние животные.txt
Вероятная тема текста: научная фантастика.txt

Анализация научная фантастика.txt
Вероятная тема текста: научная фантастика.txt

Анализация военная документалистика.txt
Вероятная тема текста: документальная литература.txt

Анализация сад и огород.txt
Вероятная тема текста: домашние животные.txt

Анализация программирование.txt
Вероятная тема текста: домашние животные.txt

Анализация кулинария.txt
Вероятная тема текста: кулинария.txt



Проблемы возникли только с документалистикой, но она и так довольная разная бывает.

#### Задача 2 (3 балла). 

Некоторые предлоги в русском языке могут управлять разными падежами (например, "я еду в Лондон" vs "я живу в Лондоне"). Давайте проанализируем эти предлоги и их падежи. Необходимо:

- составить список таких предлогов (РГ-80 вам в помощь)
- взять достаточно большой текст (можно большое художественное произведение)
- сделать морфоразбор этого текста
- Посчитать, как часто и какие падежи встречаются у слова, идущего после предлога.

Примечания: во-первых, имейте в виду, что иногда после предлога могут идти самые неожиданные вещи: "я что, должен ехать на, черт побери, северный полюс?". Во-вторых, неплохо бы учитывать отсутствие пунктуации (конечно, в норме, как нам кажется, предлог обязательно требует зависимое, но! "да иди ты на!") Эти штуки можно отсеять, если просто учитывать только заранее определенные падежи, а не считать все, какие встретились (так и None можно огрести). 

In [ ]:
# р.п   без, для, до, из, из-за, из-под, меж, между, от, с, у
# д.п   к, по
# в.п   в, за, на, о, по, под, про, с, через
# т.п   за, меж, между, над, перед, под, с
# п.п   в, на, о, по, при
# 
# меж   р.п, т.п
# между р.п, т.п
# с     р.п, в.п, т.п
# по    д.п, в.п, п.п
# в     в.п, п.п
# за    в.п, т.п
# на    в.п, п.п
# о     в.п, п.п
# под   в.п, т.п 

In [9]:
import os
from collections import Counter
from natasha import (
    Segmenter,
    NewsEmbedding,
    NewsMorphTagger,
    Doc
)

In [ ]:
segmenter = Segmenter()
embeding = NewsEmbedding()
morph_tagger = NewsMorphTagger(embeding)

with open("литература.txt") as file:
    text = file.read()

doc = Doc(text)

counter = Counter()

adp_list = ["меж", "между", "с", "по", "в", "за", "на", "о", "под"]

doc.segment(segmenter)
doc.tag_morph(morph_tagger)
for sent in doc.sents:
    iterator = iter(sent.morph.tokens)

    for token in iterator:
        # если токен — предлог из списка
        # (token.pos == "ADP" не обязательно проверять, но на всякий случай можно)
        if token.text.lower() in adp_list and token.pos == "ADP":
            try:
                # тогда смотрим на следующее слово
                # и если оно есть и у него есть падеж
                # добавляем в счетчик
                next_token = next(iterator)
                if "Case" in next_token.feats:
                    adp_case = token.text.lower() + "_" + next_token.feats["Case"]
                    counter[adp_case] += 1

                    case_ = next_token.feats["Case"] + "_all"
                    counter[case_] += 1
            except StopIteration:
                ...
    
for k, v in sorted(counter.items()):
    print(f"Количество {k} = {v}")

Количество Acc_all = 2990
Количество Dat_all = 698
Количество Gen_all = 289
Количество Ins_all = 1455
Количество Loc_all = 3314
Количество Nom_all = 10
Количество Par_all = 3
Количество в_Acc = 1294
Количество в_Ins = 1
Количество в_Loc = 2091
Количество в_Nom = 5
Количество за_Acc = 188
Количество за_Gen = 2
Количество за_Ins = 269
Количество за_Nom = 3
Количество меж_Gen = 1
Количество меж_Ins = 1
Количество между_Gen = 15
Количество между_Ins = 49
Количество на_Acc = 1425
Количество на_Gen = 3
Количество на_Ins = 1
Количество на_Loc = 881
Количество о_Acc = 17
Количество о_Ins = 1
Количество о_Loc = 336
Количество по_Acc = 15
Количество по_Dat = 698
Количество по_Loc = 6
Количество под_Acc = 48
Количество под_Ins = 119
Количество с_Acc = 3
Количество с_Gen = 268
Количество с_Ins = 1014
Количество с_Nom = 2
Количество с_Par = 3


Забавно, тут есть предлог о с инструменталисом. На самом деле это "о злом племени". ну тут уже не моя ошибка, можно просто выкинуть элементы, которые встречаются менее 10 раз.

#### Задача 3. 

Представим, что у вас есть файл с разборами conllu (можете взять любой, например, [тут](https://github.com/dialogue-evaluation/GramEval2020)). Нужно просмотреть все примеры предложений с тегом dislocated и тегом discourse: напишите скрипт, который будет читать файл, находить все такие предложения и печатать: 1) сам текст предложения 2) слово, имеющее искомый тег. Если тег не был найден в файле, нужно об этом сообщить. Постарайтесь оформить вывод таким образом, чтобы это было удобно читать. 

In [8]:
from conllu import parse_incr

In [19]:
def collect_sentence(sentence):
    collected_sentence = ""
    for token in sentence:
        collected_sentence += token["form"]
        if token["misc"] is not None:
            continue
        collected_sentence += " "
    return collected_sentence

In [30]:
with open("GramEval2020-GSD-train.conllu", "r", encoding="utf-8") as data_file:
    sentences = {}

    parser = parse_incr(data_file)
    for i, sentence in enumerate(parser):
        # if i != 0:
        #     break
        for token in sentence:
            if token['deprel'] == 'dislocated' or token['deprel'] == 'discourse':
                collected_sentence = collect_sentence(sentence)
                sentences[collected_sentence] = (token["form"], token['deprel'])
                # break чтобы одно предложение с 2+ токенами появлялось лишь однажды
                break

if len(sentences) == 0:
    print("Тегов dislocated и discourse нет в файле")
else:
    for k, v in sentences.items():
        print(f"{k}\nСлово с тегом {v[1]}: {v[0]}")

Сын чиновника, Покровский родился в 1808 г. и воспитывался в Казанской Гимназии, в которой окончил курс в 1820 г.; в 1823 г. он поступил в Казанский Университет (28-го июня), из которого вышел 28-го июня 1826 г. со степенью кандидата, после чего определился учителем средних классов Казанской Гимназии, с прикомандированием к Педагогическому Институту, состоявшему при Университете. 
Слово с тегом dislocated: Сын
Поп Григорий показывал: ``какая де на нем, архиерее, скорбь была, про тое де скорбь он, поп Григорий, не скажет, потому что за клятвою его, архиерейскою&#39;&#39;. 
Слово с тегом discourse: де


#### Задача 4.

Возьмите любой достаточно длинный (лучше новостной) текст. Любым известным инструментом извлеките именованные сущности из этого текста и выведите их списком по категориям (т.е. персоны вместе, локации вместе, организации вместе). 

In [15]:
from natasha import (
    Segmenter,
    NewsEmbedding,
    NewsMorphTagger,
    NewsNERTagger,
    MorphVocab,
    Doc
)

In [ ]:
segmenter = Segmenter()
embeding = NewsEmbedding()
morph_tagger = NewsMorphTagger(embeding)
morph_vocab = MorphVocab()
ner_tagger = NewsNERTagger(embeding)

with open("новости.txt") as file:
    text = file.read()

doc = Doc(text)
doc.segment(segmenter)
doc.tag_ner(ner_tagger)

locations, organisations, persons = [], [], []

for span in doc.spans:
    if span.type == "LOC":
        locations.append(span.text)
    elif span.type == "PER":
        persons.append(span.text)
    else:
        organisations.append(span.text)

#лемматизируем, чтобы все было в н.ф.
locations = [morph_vocab.normal_forms(word)[0].title() for word in locations]
persons = [morph_vocab.normal_forms(word)[0].title() for word in persons]
organisations = [morph_vocab.normal_forms(word)[0].title() for word in organisations]

locations = set(locations)
persons = set(persons)
organisations = set(organisations)

# хотя при лемматизации у нас получается
# Балтийского Морить и Фонд Борьбы С Коррупция
# тут надо что-то посложнее, чем просто .normal_forms
# или забить и максимум превратить списки в множества

print("Места")
for i in locations:
    print(i, end=", ")

print("\nЛюди")
for i in persons:
    print(i, end=", ")

print("\nОрганизации")
for i in organisations:
    print(i, end=", ")


Места
Балтийского Морить, Пальмира, Туапсе, Великобритания, Крым, Тюменской Область, Америка, Земля, Пермь, Амстердам, Техас, Украина, Китай, Карелия, Британия, Капитолий, Чукотка, Якутия, Афганистан, Хабаровскому Крать, Ссср, Краснокаменка, Донбасс, Феодосия, Шереметьево, Япония, Литва, Кубань, Ростовскую Область, Северной Америк, Нидерланды, Ялта, Россия, Берлин, Кабул, Сша, Фрг, Франция, Суэцком Канал, Весна, Ирак, Белоруссия, Геленджик, Суэцкого Канал, Польша, Москва, Норильск, Токио, Германия, Сочи, Австралия, Варшава, Казань, Юг, Саратовской Областить, Старомарьевка, Санкт-Петербург, Краснодарский Край, Евросоюз, Европа, Днр, Штат, Соединённых Штатах Америк, Иркутский, Лнр, 
Люди
Николя Саркози, Рашкина, Казаться, Денис Майдан, Александр Бастрыкин, Тимофей Бажен, Елены Агапов, Антониу Гутерреш, Петр Толстой, Муратов, Джо Байден, Элла Памфилов, Юлией Пересильда, Дмитрий Певцов, Меркель, Мария Захаров, Евгений Зиничев, Валентина Матвиенко, Алексей Навальный, Владимира Путин, Фрау М

#### Задача 5 (3 балла). 

А. Циммерлинг в [статье 2023 года](https://dialogue-conf.org/media/5937/zimmerlingav120.pdf) исследовал дативно-предикативные конструкции (вида "мне холодно", "мне больно" и т.п.). Возьмите любой достаточно большой датасет (можно попросить у меня корпус ГИКРЯ или создать собственный) и посчитайте аналогичные метрики на этом датасете для предикатов, которые приведены в статье. Имейте в виду, что тега "предикатив" в UD нет, потому и предлагается конкретный список слов, но слова типа "холодно"/"хорошо" могут не быть предикатами. 

In [ ]:
# your code here